# Python based analysis

This script takes trees as inputs and writes another sets of trees as outputs.<br>
The trees are filtered in between.

In [1]:
import os, sys
import numpy as np
import uproot
import pandas as pd
from tqdm import tqdm

In [2]:
#Global parameters

basedir = '../input_files/trees'
injob  = 'tree_ee_Zwindow_2018UL_Feb27'
outjob = 'tree_ee_Zwindow_2018UL_Mar12'
campaign = '2018_UL'

indir = os.path.join(basedir, injob)
outdir = os.path.join(basedir, outjob)
os.makedirs(outdir, exist_ok=True) 

In [3]:
import uproot
import pandas as pd
from tqdm.notebook import tqdm

def read_file_into_df(filepath, step_size=100000):
    with uproot.open(filepath) as tfile:
        if "myEvents" not in tfile:
            print(f"Warning: 'myEvents' tree not found in {filepath}. Skipping.")
            return pd.DataFrame()

        ttree = tfile["myEvents"]
        total_entries = ttree.num_entries

        if total_entries == 0:
            print(f"Warning: 'myEvents' tree in {filepath} is empty. Skipping.")
            return pd.DataFrame()

        total_batches = (total_entries + step_size - 1) // step_size
        dfs = []

        with tqdm(total=total_batches, unit="batch",
                  bar_format="{l_bar}{bar}| [{elapsed} < {remaining}, {n_fmt}/{total_fmt}]",
                  leave=True, ncols=100, desc="Reading", colour='green') as pbar:
            for batch in ttree.iterate(ttree.keys(), step_size=step_size, library="pd"):
                dfs.append(batch)
                pbar.update(1)

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

def write_df_into_file(df, filepath, step_size=100000):
    if df.empty:
        print(f"Warning: Attempting to write empty DataFrame to {filepath}. Skipping.")
        return

    total_entries = len(df)
    total_batches = (total_entries + step_size - 1) // step_size

    with tqdm(total=total_batches, unit="batch", 
              bar_format="{l_bar}{bar}| {percentage:3.0f}% [{elapsed} < {remaining}, {n_fmt}/{total_fmt}]",
              leave=True, ncols=100, desc="Writing", colour='green') as pbar:
        
        with uproot.recreate(filepath) as file:
            for start in range(0, total_entries, step_size):
                chunk = df.iloc[start:start + step_size]
                if start == 0: file["myEvents"] = {col: chunk[col].to_numpy() for col in df.columns}
                else:          file["myEvents"].extend({col: chunk[col].to_numpy() for col in df.columns})
                pbar.update(1)

print('Functions loaded.')

Functions loaded.


In [4]:
%%time

list_of_files = os.listdir(indir)
list_empty_input = []
list_empty_output = []

for f in list_of_files:

    #if not 'QCD' in f: continue
    if not f.endswith('.root'): continue
    #if 'SingleMuon' not in f: continue

    #Step1: Prepare the dataframe
    print(f'\nProcessing file: {f}')
    filepath = os.path.join(indir, f)
    sample = f.split("_")[1]
    subsample = "_".join(f.split("_")[2:])
       
    outfile = os.path.join(outdir, f)
    #if os.path.exists(outfile): continue
    #if 'WJets' not in f: continue

    df = read_file_into_df(filepath)
    if df.empty : 
        #print(f"\033[0;31mWarning: Empty input file: {filepath}\033[0m\n")
        list_empty_input.append(filepath)
        continue

    '''
    #Creating working points on the dataframe using the NN.
    df['flag_qcd'] = (
        ((df['channel'].isin([0, 1])) & (df['nnscore_qcd_vlldmu_200_800']  < 0.30)) |
        ((df['channel'].isin([2, 3])) & (df['nnscore_qcd_vlldele_200_800'] < 0.30))
    )

    df['flag_topregion'] = (
        ((df['channel'].isin([0, 1])) & (df['nnscore_qcd_vlldmu_200_800']  > 0.50) & (df['nnscore_qcd_vlldmu_200_800']  < 0.70)) |
        ((df['channel'].isin([2, 3])) & (df['nnscore_qcd_vlldmu_200_800']  > 0.50) & (df['nnscore_qcd_vlldele_200_800'] < 0.70))
    )
    
    df['flag_searchregion'] = (
        ((df['channel'].isin([0, 1])) & (df['nnscore_qcd_vlldmu_200_800']  > 0.70)) |
        ((df['channel'].isin([2, 3])) & (df['nnscore_qcd_vlldele_200_800'] > 0.70))
    )

    nnscore = f'nnscore_qcd_vlld_{campaign.replace("_UL", "")}'
    df['flag_searchregion'] = df[nnscore] > 0.70
    df['flag_val'] = (df[nnscore] > 0.50) & (df[nnscore] < 0.7)
    df['flag_qcd'] = (df[nnscore] < 0.30)

    # Put any cuts here on dataframe as you like
    # Define conditions as strings
    dy_veto = 'not (channel == 3 and 76 < dilep_mass < 106)'

    #Step1: Controlling QCD:
    qcd_cr = f'{dy_veto} and flag_qcd and 0.02<lep0_iso<0.15 and lep0_sip3d>5'
    qcd_vr = f'{dy_veto} and flag_qcd and 0.02<lep0_iso<0.15 and lep0_sip3d<5'

    #Step2: Controlling Drell-Yan:
    dycr  = f'76<dilep_mass<106  and dilep_ptratio > 0.7'

    tight_sip3d =  'lep0_sip3d<5 and lep1_sip3d<10'
    qcd_veto = f'{tight_sip3d} and {dy_veto} and not flag_qcd and HT>50'

    #Step3: Search region
    search_region = f'{qcd_veto} and flag_searchregion'
    
    #Step5: Controlling TTbar:
    top_vr = f'{qcd_veto} and flag_val'
    #top_cr = f'{qcd_veto} and flag_searchregion and nbjet>0'
    top_cr = f'{search_region} and nnscore_ttbar_vlld < 0.40'

    #Step5: Signal regions:
    #sig_region = f'{search_region} and and nbjet == 0'
    sig_region = f'{search_region} and nnscore_ttbar_vlld > 0.70'
    '''

    dy_cr = f'76<dilep_mass<106 and lep0_sip3d<5 and lep1_sip3d<10 and dilep_ptratio>0.7'
    
    #------------------------------
    # Final event selection:
    event_selection = dy_cr
    #------------------------------

    # Use df.query with the combined condition
    df_filtered = df.query(event_selection)
    
    if df_filtered.empty: list_empty_output.append(f'{sample}_{subsample}')

    nbefore = len(df)
    nafter = len(df_filtered)

    frac = 0
    if nbefore != 0: frac = nafter*100/nbefore

    #display(df_filtered)

    write_df_into_file(df_filtered, outfile)
    print(f'\033[1;32mFile written: {outfile}\033[0m ({nbefore} -> {nafter}, {frac:.2f}%)\n')
    #break #file

print(f'\n\033[1;33mSummary:\033[0m')
print(f'\n\033[33mEmpty input: {list_empty_input}\033[0m')
print(f'\n\033[31mEmpty output: {list_empty_output}\033[0m\n')


Processing file: tree_DYJetsToLL_M10to50.root


Reading:   0%|                                                                    | [00:00 < ?, 0/1]

Writing:   0%|                                                               |   0% [00:00 < ?, 0/1]

File written: ../input_files/trees/tree_ee_Zwindow_2018UL_Mar12/tree_DYJetsToLL_M10to50.root (8 -> 3, 37.50%)


Processing file: tree_DYJetsToLL_M50.root


Reading:   0%|                                                                  | [00:00 < ?, 0/133]

...

Writing:   0%|                                                               |   0% [00:00 < ?, 0/6]

File written: ../input_files/trees/tree_ee_Zwindow_2018UL_Mar12/tree_ZZ_ZZTo2Q2L.root (1959773 -> 568695, 29.02%)


Processing file: tree_ZZ_ZZTo2Q2Nu.root

Processing file: tree_ZZ_ZZTo4L.root


Reading:   0%|                                                                   | [00:00 < ?, 0/24]

Writing:   0%|                                                              |   0% [00:00 < ?, 0/12]

File written: ../input_files/trees/tree_ee_Zwindow_2018UL_Mar12/tree_ZZ_ZZTo4L.root (2373974 -> 1163110, 48.99%)


Summary:

Empty input: ['../input_files/trees/tree_ee_Zwindow_2018UL_Feb27/tree_QCD_EMEnriched_15to20.root', '../input_files/trees/tree_ee_Zwindow_2018UL_Feb27/tree_QCD_EMEnriched_170to300.root', '../input_files/trees/tree_ee_Zwindow_2018UL_Feb27/tree_QCD_EMEnriched_20to30.root', '../input_files/trees/tree_ee_Zwindow_2018UL_Feb27/tree_QCD_EMEnriched_30to50.root', '../input_files/trees/tree_ee_Zwindow_2018UL_Feb27/tree_QCD_EMEnriched_50to80.root', '../input_files/trees/tree_ee_Zwindow_2018UL_Feb27/tree_QCD_EMEnriched_80to120.root', '../input_files/trees/tree_ee_Zwindow_2018UL_Feb27/tree_QCD_MuEnriched_20to30.root', '../input_files/trees/tree_ee_Zwindow_2018UL_Feb27/tree_QCD_MuEnriched_30to50.root', '../input_files/trees/tree_ee_Zwindow_2018UL_Feb27/tree_QCD_MuEnriched_80to120.root', '../input_files/trees/tree_ee_Zwindow_2018UL_Feb27/tree_ZZ_ZZTo2Q2Nu.root']

Empty output: [

In [5]:
%%time

run_here = False

channels = ['mm', 'me', 'em', 'ee']
for channel_ in channels:
    jobname_ = outjob
    campaign_ = campaign
    dump_ = outjob.replace('tree', 'hist')+'_'+channel_
    arguments = f'extractHistsFromTrees.C("{jobname_}", "{dump_}", "{campaign_}", "{channel_}")'
    command = f"root -q -b -l '{arguments}'"
    print(f'\033[93m\nRunnning .. \n{command}\033[0m\n')

    if run_here: os.system(command)
    else: print('Run it yourself:')
    
print('Done!')


Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("tree_ee_Zwindow_2018UL_Mar12", "hist_ee_Zwindow_2018UL_Mar12_mm", "2018_UL", "mm")'

Run it yourself:

Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("tree_ee_Zwindow_2018UL_Mar12", "hist_ee_Zwindow_2018UL_Mar12_me", "2018_UL", "me")'

Run it yourself:

Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("tree_ee_Zwindow_2018UL_Mar12", "hist_ee_Zwindow_2018UL_Mar12_em", "2018_UL", "em")'

Run it yourself:

Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("tree_ee_Zwindow_2018UL_Mar12", "hist_ee_Zwindow_2018UL_Mar12_ee", "2018_UL", "ee")'

Run it yourself:
Done!
CPU times: user 162 μs, sys: 3 μs, total: 165 μs
Wall time: 158 μs
